In [1]:
#import packages
import torch
import copy
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('../') 
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt
import warnings
from tasks_celebA import CelebADataset, CelebNet, fast_adapt
warnings.filterwarnings("ignore")
from maml_model import MamlModel
%matplotlib inline

In [2]:
alpha=1
beta=0.001
shots=100
adaptation_steps=1
meta_batch_size = 32
num_iterations = 20000
ordered_pixels = 'True'
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
celebdistr = CelebADataset('train' , device)
batch = celebdistr.sample_tasks(meta_batch_size)
train_inputs = celebdistr.sample_inputs(shots, ordered_pixels).to(device)
targets = batch[0](train_inputs)
targets = batch[0]

In [4]:
num_hidden_layers = [128, 128, 128, 128, 128]
model = MamlModel(celebdistr.num_inputs, celebdistr.num_outputs, num_hidden_layers,128, device)
model.to(device)
loss=nn.MSELoss()

opt = torch.optim.Adam(model.weights + model.biases + [model.task_context], beta)

In [5]:
for iteration in range(num_iterations):
    opt.zero_grad()
    meta_train_error = 0.0
    #sample a meta-batch
    target_functions = celebdistr.sample_tasks(meta_batch_size)
    #plt.figure(figsize=(4, 4))
    #plt.imshow(batch_images[0].cpu().numpy())
   # plt.show()
    init_weights=[w.clone() for w in model.parameters()]
    for task in range(meta_batch_size):
        
        x = celebdistr.sample_inputs(shots*2, ordered_pixels).to(device) #get both inner/outer inputs
        y = target_functions[task](x)
       # print(x.shape, y.shape)
        evaluation_error= fast_adapt(x, y, alpha, model, loss, adaptation_steps, shots, device)
        for p,t in zip(model.parameters(), init_weights): # backpropagate w.r.t. init weights
            p.data = t
        evaluation_error.backward()
        
        meta_train_error += evaluation_error.item()
        
    print('\n')
    print('Iteration', iteration)
    print('Meta Train Error', meta_train_error / meta_batch_size)
         
    for p in model.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
            #print(torch.sum(p), torch.sum(p.grad))
    opt.step()



Iteration 0
Meta Train Error 0.19397840916644782


Iteration 1
Meta Train Error 0.14076847536489367


Iteration 2
Meta Train Error 0.1166104951989837


Iteration 3
Meta Train Error 0.0977309254813008


Iteration 4
Meta Train Error 0.11339416081318632


Iteration 5
Meta Train Error 0.0973015182535164


Iteration 6
Meta Train Error 0.10849039285676554


Iteration 7
Meta Train Error 0.10304331986117177


Iteration 8
Meta Train Error 0.11223482654895633


Iteration 9
Meta Train Error 0.11497364373644814


Iteration 10
Meta Train Error 0.11198105811490677


Iteration 11
Meta Train Error 0.12080697930650786


Iteration 12
Meta Train Error 0.10738674510503188


Iteration 13
Meta Train Error 0.1427775756455958


Iteration 14
Meta Train Error 0.11983323941240087


Iteration 15
Meta Train Error 0.10551184887299314


Iteration 16
Meta Train Error 0.12576836068183184


Iteration 17
Meta Train Error 0.098627971339738


Iteration 18
Meta Train Error 0.10047680776915513


Iteration 19
Meta Train Er



Iteration 158
Meta Train Error 0.11937955860048532


Iteration 159
Meta Train Error 0.1175727744557662


Iteration 160
Meta Train Error 0.1177098483312875


Iteration 161
Meta Train Error 0.09755577699979767


Iteration 162
Meta Train Error 0.09470228613645304


Iteration 163
Meta Train Error 0.11471454169077333


Iteration 164
Meta Train Error 0.09755730605684221


Iteration 165
Meta Train Error 0.09054422228655312


Iteration 166
Meta Train Error 0.10487676039338112


Iteration 167
Meta Train Error 0.1256393062067218


Iteration 168
Meta Train Error 0.08614224764460232


Iteration 169
Meta Train Error 0.10581121733412147


Iteration 170
Meta Train Error 0.12706656649243087


Iteration 171
Meta Train Error 0.12894490403414238


Iteration 172
Meta Train Error 0.11277348472503945


Iteration 173
Meta Train Error 0.12012445880100131


Iteration 174
Meta Train Error 0.09874957252759486


Iteration 175
Meta Train Error 0.10376092192018405


Iteration 176
Meta Train Error 0.09635758279182



Iteration 314
Meta Train Error 0.08876252146728802


Iteration 315
Meta Train Error 0.12454000549041666


Iteration 316
Meta Train Error 0.09760047908639535


Iteration 317
Meta Train Error 0.11269592843018472


Iteration 318
Meta Train Error 0.12100475677289069


Iteration 319
Meta Train Error 0.09236893155321013


Iteration 320
Meta Train Error 0.09799748042132705


Iteration 321
Meta Train Error 0.11048061290057376


Iteration 322
Meta Train Error 0.0974228882114403


Iteration 323
Meta Train Error 0.11347127635963261


Iteration 324
Meta Train Error 0.09860123306862079


Iteration 325
Meta Train Error 0.10613755925442092


Iteration 326
Meta Train Error 0.11344296555034816


Iteration 327
Meta Train Error 0.11894858058076352


Iteration 328
Meta Train Error 0.09954963956261054


Iteration 329
Meta Train Error 0.0942534763016738


Iteration 330
Meta Train Error 0.10348248016089201


Iteration 331
Meta Train Error 0.10619188100099564


Iteration 332
Meta Train Error 0.0944235556235



Iteration 470
Meta Train Error 0.11616313576814719


Iteration 471
Meta Train Error 0.12063223187578842


Iteration 472
Meta Train Error 0.11199302697787061


Iteration 473
Meta Train Error 0.11262555426219478


Iteration 474
Meta Train Error 0.10453856470121536


Iteration 475
Meta Train Error 0.10537023487268016


Iteration 476
Meta Train Error 0.12083565536886454


Iteration 477
Meta Train Error 0.08189920657605398


Iteration 478
Meta Train Error 0.10003059983137064


Iteration 479
Meta Train Error 0.08865627832710743


Iteration 480
Meta Train Error 0.1312445643125102


Iteration 481
Meta Train Error 0.11649056506576017


Iteration 482
Meta Train Error 0.0979012492171023


Iteration 483
Meta Train Error 0.11167846160242334


Iteration 484
Meta Train Error 0.10458683129400015


Iteration 485
Meta Train Error 0.10515522552304901


Iteration 486
Meta Train Error 0.11679865000769496


Iteration 487
Meta Train Error 0.10495463153347373


Iteration 488
Meta Train Error 0.0974403846776



Iteration 626
Meta Train Error 0.1238137100590393


Iteration 627
Meta Train Error 0.09699780389200896


Iteration 628
Meta Train Error 0.10384362126933411


Iteration 629
Meta Train Error 0.1042135609604884


Iteration 630
Meta Train Error 0.10877179172530305


Iteration 631
Meta Train Error 0.09334846751880832


Iteration 632
Meta Train Error 0.11749288748251274


Iteration 633
Meta Train Error 0.11559758521616459


Iteration 634
Meta Train Error 0.11211000598268583


Iteration 635
Meta Train Error 0.09113184816669673


Iteration 636
Meta Train Error 0.12028946427744813


Iteration 637
Meta Train Error 0.1149060327661573


Iteration 638
Meta Train Error 0.09129838661465328


Iteration 639
Meta Train Error 0.10634338989621028


Iteration 640
Meta Train Error 0.09333420341135934


Iteration 641
Meta Train Error 0.10311559934052639


Iteration 642
Meta Train Error 0.12008629148476757


Iteration 643
Meta Train Error 0.10822904904489405


Iteration 644
Meta Train Error 0.12054552935296

...



Iteration 19320
Meta Train Error 0.09096435518586077


Iteration 19321
Meta Train Error 0.10226753211463802


Iteration 19322
Meta Train Error 0.10378010061685927


Iteration 19323
Meta Train Error 0.10643486377375666


Iteration 19324
Meta Train Error 0.12342992069898173


Iteration 19325
Meta Train Error 0.09531711769523099


Iteration 19326
Meta Train Error 0.10186427825829014


Iteration 19327
Meta Train Error 0.11405510304030031


Iteration 19328
Meta Train Error 0.09769316401798278


Iteration 19329
Meta Train Error 0.11654636785533512


Iteration 19330
Meta Train Error 0.10210578655824065


Iteration 19331
Meta Train Error 0.12305477843619883


Iteration 19332
Meta Train Error 0.1027847685618326


Iteration 19333
Meta Train Error 0.11867803930363152


Iteration 19334
Meta Train Error 0.11322393448790535


Iteration 19335
Meta Train Error 0.11542726453626528


Iteration 19336
Meta Train Error 0.11176152038387954


Iteration 19337
Meta Train Error 0.1040545049472712


Iteration 



Iteration 19470
Meta Train Error 0.12091012531891465


Iteration 19471
Meta Train Error 0.12016607541590929


Iteration 19472
Meta Train Error 0.1203378222126048


Iteration 19473
Meta Train Error 0.1031951608019881


Iteration 19474
Meta Train Error 0.10616006139025558


Iteration 19475
Meta Train Error 0.09029753046343103


Iteration 19476
Meta Train Error 0.1154647067305632


Iteration 19477
Meta Train Error 0.10355980953318067


Iteration 19478
Meta Train Error 0.11342723402776755


Iteration 19479
Meta Train Error 0.11187171138590202


Iteration 19480
Meta Train Error 0.09541673472267576


Iteration 19481
Meta Train Error 0.11934905004454777


Iteration 19482
Meta Train Error 0.12749349587829784


Iteration 19483
Meta Train Error 0.1257982215611264


Iteration 19484
Meta Train Error 0.10927966043527704


Iteration 19485
Meta Train Error 0.1046368254465051


Iteration 19486
Meta Train Error 0.10180689430126222


Iteration 19487
Meta Train Error 0.10810653482622001


Iteration 194



Iteration 19620
Meta Train Error 0.11588649568147957


Iteration 19621
Meta Train Error 0.13581221393542364


Iteration 19622
Meta Train Error 0.10858683497644961


Iteration 19623
Meta Train Error 0.09197728968865704


Iteration 19624
Meta Train Error 0.11465693320496939


Iteration 19625
Meta Train Error 0.10764247036422603


Iteration 19626
Meta Train Error 0.11918611731380224


Iteration 19627
Meta Train Error 0.09836545868893154


Iteration 19628
Meta Train Error 0.12695068481843919


Iteration 19629
Meta Train Error 0.13264768716180697


Iteration 19630
Meta Train Error 0.09592473751399666


Iteration 19631
Meta Train Error 0.09643935540225357


Iteration 19632
Meta Train Error 0.10042100292048417


Iteration 19633
Meta Train Error 0.11581733130151406


Iteration 19634
Meta Train Error 0.10611278706346639


Iteration 19635
Meta Train Error 0.10151076038891915


Iteration 19636
Meta Train Error 0.12372310250066221


Iteration 19637
Meta Train Error 0.10388344392413273


Iteratio



Iteration 19770
Meta Train Error 0.12797930592205375


Iteration 19771
Meta Train Error 0.1337060366058722


Iteration 19772
Meta Train Error 0.1001374809857225


Iteration 19773
Meta Train Error 0.11180834862170741


Iteration 19774
Meta Train Error 0.07773157715564594


Iteration 19775
Meta Train Error 0.10946856840746477


Iteration 19776
Meta Train Error 0.11388428015925456


Iteration 19777
Meta Train Error 0.10538248548982665


Iteration 19778
Meta Train Error 0.10736618441296741


Iteration 19779
Meta Train Error 0.09302468097303063


Iteration 19780
Meta Train Error 0.11232564324745908


Iteration 19781
Meta Train Error 0.12380364688578993


Iteration 19782
Meta Train Error 0.12647268842556514


Iteration 19783
Meta Train Error 0.11487192232743837


Iteration 19784
Meta Train Error 0.09305991284054471


Iteration 19785
Meta Train Error 0.1069734938791953


Iteration 19786
Meta Train Error 0.10113491048105061


Iteration 19787
Meta Train Error 0.10849882801994681


Iteration 1



Iteration 19920
Meta Train Error 0.12802691062097438


Iteration 19921
Meta Train Error 0.10714543746144045


Iteration 19922
Meta Train Error 0.10517379979137331


Iteration 19923
Meta Train Error 0.10438372689532116


Iteration 19924
Meta Train Error 0.0797125492390478


Iteration 19925
Meta Train Error 0.09060040087206289


Iteration 19926
Meta Train Error 0.11546451854519546


Iteration 19927
Meta Train Error 0.11372823745477945


Iteration 19928
Meta Train Error 0.1094077640445903


Iteration 19929
Meta Train Error 0.09556387343036477


Iteration 19930
Meta Train Error 0.11739813379244879


Iteration 19931
Meta Train Error 0.1087041223727283


Iteration 19932
Meta Train Error 0.11815708153881133


Iteration 19933
Meta Train Error 0.10705251162289642


Iteration 19934
Meta Train Error 0.1106422345910687


Iteration 19935
Meta Train Error 0.10942907625576481


Iteration 19936
Meta Train Error 0.11041944217868149


Iteration 19937
Meta Train Error 0.11674413015134633


Iteration 19